In [1]:
import _MEWtools as mt
import multiprocessing

## Get baseline time for CPU Intensive Workload

In [2]:
iterations = 5

def slow_worker():
    for i in range(8000):
        square = i**i

    print('Done')

In [3]:
import time

t0 = time.time()

for i in range(iterations):
    slow_worker()

t1 = time.time()

total = t1-t0
display(f'Total Execution Time: {total} seconds')

Done
Done
Done
Done
Done


'Total Execution Time: 11.981179237365723 seconds'

## Perform same operation with multiprocessing

In [4]:
t0 = time.time()

pool = multiprocessing.Pool(iterations)
for i in range(iterations):
    pool.apply_async(slow_worker)
    
pool.close()
pool.join()

t1 = time.time()

total = t1-t0
display(f'Total Execution Time: {total} seconds')

Done
Done
Done
Done
Done


'Total Execution Time: 2.8560779094696045 seconds'

## Load an interior structure from a config file and populate a MEWTools Satellite

In [1]:
import utils

sample_sat = utils.import_structure('Sample')

In [7]:
from os import listdir

listdir('./structures')

['Sample.yml']

In [4]:
structures = utils.list_structures()

Sample


In [5]:
structures

['Sample']